# Scraping on Football Data

I'll use a scraping tecnique in order to obtain data from the famous site Transfermarkt TM.
In particular I'm going to download from the website a table with all the data about a specific football player's matches in a season.

### Package Import
First of all I'm going to import all the packages I need. I'll use **Requests** and **BeautifulSoup** from **bs4** in order to perform the scraping itself, and then **Pandas** in order to manage data into Dataframes.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Scraping Class

I won't only write a script to perform scraping, but I'll use a Class, so I first create the object and then download the data.

I used some Italian words in the code, they're just variables' and functions' names.

In [3]:
class scraper():
    def __init__(self):
        pass
    def url_importer(self,code):
    #The function imports the url and prepares it to be used in scraping
        import requests
        from bs4 import BeautifulSoup
        import pandas as pd
        import numpy as np
        headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        URL = code
        page = requests.get(URL,headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")
        return(soup)
    def find_tables(self,table,stopword):
    #The Function divides the url by defined word and separates it using the same stop word in order to have the html code of each table
    # in the page
        results = table.find_all(stopword)
        tabelle=str(results)
        tabelle=tabelle.split(stopword)
        return(tabelle)
    def find_tabella_definitiva(self,tabella,testa):
    #The function finds the table in which data are stored
    #I already know I'm searching for the last table in the page, but if you need an other table you'll need to
    #modify this part of the code
        for el in tabella:
            if testa in el:
                tabella_definitiva=el
        self.tabella_definitiva=tabella_definitiva
    #Creation of a dataframe with all the matches played by the player
    def dataframe_Presenze(self):
        tabella=self.tabella_definitiva
        tabella =BeautifulSoup(tabella)
        dati=tabella.find_all("tr", class_="")
        giornate = pd.DataFrame(columns=["Match_Number","Date","Home","Away","Result","Role","Gol","Assist","Owngoal","Yellow card","Double yellow card","Red card","Sub in","Sub out"])
        for partita in dati:
            colonne=partita.find_all("td", class_="zentriert")
            prestazione=[]
            for elemento in colonne:
                if elemento.text!="":
                    prestazione.append(elemento.text.replace("\n"," ").strip(" "))
                    if len(prestazione)> 5:
                        prestazione[5]=prestazione[5].replace("\xa0"," CAPITAIN")
                    if len(prestazione)>13:
                        prestazione[13]=prestazione[13].replace("\xa0"," INJURY")
                elif elemento.text == "":
                    squadra=str(elemento.img).split('"')
                    if len(squadra)>1:
                        prestazione.append(squadra[1])
                    else :
                        prestazione.append("0")
    
            prestazione=pd.Series(prestazione, index=giornate.columns)
            giornate=giornate.append(prestazione,ignore_index=True)
            self.giocate=giornate
        return(giornate)
    #Creation of a dataframe with all the matches missed by the player because of an injury
    def dataframe_Assenze(self):
        tabella=self.tabella_definitiva
        tabella =BeautifulSoup(tabella)
        dati=tabella.find_all("tr", class_="bg_rot_20")
        giornate = pd.DataFrame(columns=["Match_Number","Date","Home","Away","Result","Role","Gol","Assist","Owngoal","Yellow card","Double yellow card","Red card","Sub in","Sub out"])
        leftovers=["0","0","0","0","0","0","0","0"]
        for partita in dati:
            colonne=partita.find_all("td", class_="zentriert")
            prestazione=[]
            for elemento in colonne:
                if elemento.text!="":
                    prestazione.append(elemento.text.replace("\n"," ").strip(" "))
                    if len(prestazione)>5:
                        prestazione[5]=prestazione[5].replace("\xa0"," ").strip(" ")
                elif elemento.text == "":
                    squadra=str(elemento.img).split('"')
                    if len(squadra)>1:
                        prestazione.append(squadra[1])
            prestazione=prestazione+leftovers
            prestazione=pd.Series(prestazione, index=giornate.columns)
            giornate=giornate.append(prestazione,ignore_index=True)
            self.assenze=giornate
        return(giornate)
    #Creation of a dataframe with all the matches missed by the player because he wasn't in the lineup
    def dataframe_panchine(self):
        tabella=self.tabella_definitiva
        tabella =BeautifulSoup(tabella)
        dati=tabella.find_all("tr", class_="bg_gelb_20")
        giornate = pd.DataFrame(columns=["Match_Number","Date","Home","Away","Result","Role","Gol","Assist","Owngoal","Yellow card","Double yellow card","Red card","Sub in","Sub out"])
        leftovers=["0","0","0","0","0","0","0","0"]
        for partita in dati:
            colonne=partita.find_all("td", class_="zentriert")
            prestazione=[]
            for elemento in colonne:
                if elemento.text!="":
                    prestazione.append(elemento.text.replace("\n"," ").strip(" "))
                    if len(prestazione)>5:
                        prestazione[5]=prestazione[5].replace("\xa0"," ").strip(" ")
                elif elemento.text == "":
                    squadra=str(elemento.img).split('"')
                    if len(squadra)>1:
                        prestazione.append(squadra[1])
            prestazione=prestazione+leftovers
            prestazione=pd.Series(prestazione, index=giornate.columns)
            giornate=giornate.append(prestazione,ignore_index=True)
            self.panchine=giornate
        return(giornate)
    #Creation of a dataframe with all the matches in the current season 
    def Create_Stagione(self):
        presenze=self.giocate
        assenze=self.assenze
        panchine=self.panchine
        totale=presenze.append(assenze)
        totale=totale.append(panchine)
        totale["Match_Number"] = pd.to_numeric(totale["Match_Number"])
        totale=totale.sort_values(by="Match_Number",axis=0)
        self.totale=totale
        return(totale)
    #Creation of a dataframe with a single line counting all the events in the season
    def crea_totale(self):
        Stagione=self.totale
        numerogol=0
        numeroassist=0
        numeroautogol=0
        numerogialli=0
        numerodoppigialli=0
        numerorossi=0
        numeropanchine=0
        numeroinfortuni=0
        for index, row in Stagione.iterrows():
            Summary=list()
            if int(row['Gol'])>0:
                numerogol+=int(row["Gol"])
            if int(row['Assist'])>0:
                numerogol+=int(row["Assist"])
            if int(row['Owngoal'])>0:
             numeroautogol+=int(row["Owngoal"])
            if row['Yellow card']!='0':
                numerogialli+=1
            if row['Double yellow card']!='0':
                numerodoppigialli+=1
                numerogialli+=1 
            if row['Red card']!='0':
                numerorossi+=1
            if row['Role']=='in panchina':
                numeropanchine+=1
            if len(row['Role'])>len('in panchina'):
                numeroinfortuni+=1
            Summary.extend((numerogol,numeroassist,numeroautogol,numerogialli,numerodoppigialli,numerorossi,numeropanchine,numeroinfortuni))
            somma = pd.DataFrame(columns=["Gol","Assist","Owngoal","Yellow card","Doube Yellow card","Red card","Bench","Injury"])
            Summary=pd.Series(Summary, index=somma.columns)
            somma=somma.append(Summary,ignore_index=True)
            somma.index=['Total']
            self.Summary=Summary
        return(somma)
    #Creation of a dataframe with a single line counting all the events in the season divided by the overall number of matches
    def numeri_su_partite(self):
        medie=list()
        Summary=self.Summary
        Stagione=self.totale
        for el in Summary:
            m=int(el)/int(max(Stagione["Match_Number"]))
            medie.append(m)
        vettore = pd.DataFrame(columns=["Gol","Assist","Owngoal","Yellow card","Doube Yellow card","Red card","Bench","Injury"])
        medie=pd.Series(medie, index=vettore.columns)
        vettore=vettore.append(medie,ignore_index=True)
        vettore.index=['Number_Matches']
        return(vettore)


## Class Using

I'm using the class I have created to scrape some data from the website.

I'll start from calling the class object and creating a variable with the URL of the page.

In [4]:
Scrape=scraper()
codice='https://www.transfermarkt.it/matthijs-de-ligt/leistungsdaten/spieler/326031/saison/2021/plus/1'

Second step is searching the data's location in the code.
I'm searching for "Serie A" beacuse I'm using a player from the Italian league, but you can use the code on other players from other leagues, the only problem is that the code can fail because of differences in the web page, like a different location of the data or a different number of columns. 

In [5]:
codice_pagina=Scrape.url_importer(codice)

table=Scrape.find_tables(codice_pagina,"tbody")

tabella_definitiva=Scrape.find_tabella_definitiva(table,"serie-a")

I'm starting to create a dataframe with matches in which the player was on the field.

In [7]:
data_giocate=Scrape.dataframe_Presenze()
print(data_giocate)

   Match_Number         Date                 Home             Away Result  \
0             1  22/ago/2021       Udinese Calcio      Juventus FC    2:2   
1             2  28/ago/2021          Juventus FC        Empoli FC    0:1   
2             3  11/set/2021           SSC Napoli      Juventus FC    2:1   
3             5  22/set/2021        Spezia Calcio      Juventus FC    2:3   
4             6  26/set/2021          Juventus FC     UC Sampdoria    3:2   
5             7  02/ott/2021            Torino FC      Juventus FC    0:1   
6            10  27/ott/2021          Juventus FC      US Sassuolo    1:2   
7            12  06/nov/2021          Juventus FC   ACF Fiorentina    1:0   
8            13  20/nov/2021             SS Lazio      Juventus FC    0:2   
9            14  27/nov/2021          Juventus FC         Atalanta    0:1   
10           15  30/nov/2021  US Salernitana 1919      Juventus FC    0:2   
11           16  05/dic/2021          Juventus FC        Genoa CFC    2:0   

Now I'll do the same with the matches missed due to injury.

In [8]:
data_assenza=Scrape.dataframe_Assenze()
print(data_assenza)

  Match_Number         Date         Home     Away Result  \
0            8  17/ott/2021  Juventus FC  AS Roma    1:0   

                      Role Gol Assist Owngoal Yellow card Double yellow card  \
0  Problema agli adduttori   0      0       0           0                  0   

  Red card Sub in Sub out  
0        0      0       0  


In the end I'll display the matches in which he wasn't sent on the field by the manager.

In [9]:
data_panchine=Scrape.dataframe_panchine()
print(data_panchine)

  Match_Number         Date           Home         Away Result         Role  \
0            4  19/set/2021    Juventus FC     AC Milan    1:1  in panchina   
1            9  24/ott/2021          Inter  Juventus FC    1:1  in panchina   
2           11  30/ott/2021  Hellas Verona  Juventus FC    2:1  in panchina   
3           23  23/gen/2022       AC Milan  Juventus FC    0:0  in panchina   

  Gol Assist Owngoal Yellow card Double yellow card Red card Sub in Sub out  
0   0      0       0           0                  0        0      0       0  
1   0      0       0           0                  0        0      0       0  
2   0      0       0           0                  0        0      0       0  
3   0      0       0           0                  0        0      0       0  


The very last pice of code creates a complete dataframe, with all the matches in the current season.

In [10]:
stagione=Scrape.Create_Stagione()
print(stagione)

    Match_Number         Date                 Home             Away Result  \
0              1  22/ago/2021       Udinese Calcio      Juventus FC    2:2   
1              2  28/ago/2021          Juventus FC        Empoli FC    0:1   
2              3  11/set/2021           SSC Napoli      Juventus FC    2:1   
0              4  19/set/2021          Juventus FC         AC Milan    1:1   
3              5  22/set/2021        Spezia Calcio      Juventus FC    2:3   
4              6  26/set/2021          Juventus FC     UC Sampdoria    3:2   
5              7  02/ott/2021            Torino FC      Juventus FC    0:1   
0              8  17/ott/2021          Juventus FC          AS Roma    1:0   
1              9  24/ott/2021                Inter      Juventus FC    1:1   
6             10  27/ott/2021          Juventus FC      US Sassuolo    1:2   
2             11  30/ott/2021        Hellas Verona      Juventus FC    2:1   
7             12  06/nov/2021          Juventus FC   ACF Fiorent

Now I'll use two functions:  
    1) To create a sigle-line dataframe with the sum of all statistics of the player;  
    2) To divide the sum of the statistics over the overall number of matches in the season so far.

In [11]:
tot=Scrape.crea_totale()
print(tot)

      Gol Assist Owngoal Yellow card Doube Yellow card Red card Bench Injury
Total   1      0       0           3                 1        0     4      1


In [13]:
div=Scrape.numeri_su_partite()
print(div)

                 Gol  Assist  Owngoal  Yellow card  Doube Yellow card  \
Number_Matches  0.04     0.0      0.0         0.12               0.04   

                Red card  Bench  Injury  
Number_Matches       0.0   0.16    0.04  


I used some functions, but I didn't need to pass arguments to many of them, that's because in the object I used Inheritance so this variables,data and dataframe could pass from a function to the succeding, in this way the class is much easier to use.  

As you may have noticed some functions didn't require an argument: that's due to Inheritance.  
It allows to pass data,variables and entitis through functions belonging to the same object, in this way my class can work without asking for parameters to the user when not needed.